In [5]:
spark.stop()

# Set paths

DEVEL_SRC must contain the directory use-cases and pykhaos

In [1]:
import os, sys
import datetime as dt
DEVEL_SRC = os.path.join(os.environ.get('BDA_USER_HOME', ''), "src", "devel")
if DEVEL_SRC not in sys.path:
    sys.path.append(DEVEL_SRC)

USECASES_SRC = os.path.join(DEVEL_SRC, "use-cases") # TODO when '-' is removed from name, remove also this line and adapt imports 
if USECASES_SRC not in sys.path: 
    sys.path.append(USECASES_SRC)
    
# AMDOCS_SRC = os.path.join(DEVEL_SRC, "amdocs_informational_dataset") # TODO when - is removed, remove also this line and adapt imports
# if AMDOCS_SRC not in sys.path: 
#     sys.path.append(AMDOCS_SRC)
    
import pykhaos.utils.custom_logger as clogger
logging_file = os.path.join(os.environ.get('BDA_USER_HOME', ''), "logging",
                                    "out_" + dt.datetime.now().strftime("%Y%m%d_%H%M%S") + ".log")
logger = clogger.configure_logger(log_filename=logging_file, std_channel=sys.stderr, logger_name="")
logger.info("Logging to file {}".format(logging_file))    
        
from project.project_generic import Project

import pykhaos.utils.notebooks as nb

import time
start_time = time.time()

RUNNING_FROM_NOTEBOOK = nb.isnotebook()
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
if RUNNING_FROM_NOTEBOOK:
    %load_ext autoreload
    %autoreload 2
    %matplotlib inline  
    EXTERNAL_LIB = os.path.join(os.environ.get('BDA_USER_HOME', ''), "lib", "external_libs")
    if EXTERNAL_LIB not in sys.path:
        sys.path.append(EXTERNAL_LIB)
    # feel free from commenting this line and the other ones that begin with "%%notify" if you do not have 
    # the extension installed or copy de lib from /var/SP/data/home/csanc109/lib/external_libs/jupyternotify/
    %load_ext jupyternotify 
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

20191226-084704 [INFO ] Logging to file /var/SP/data/home/csanc109/logging/out_20191226_084704.log


<IPython.core.display.Javascript object>

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
from pyspark.sql import Row, DataFrame, Column, Window
from pyspark.sql.types import DoubleType, StringType, IntegerType, DateType, ArrayType
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer, VectorAssembler, SQLTransformer, OneHotEncoder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.sql.functions import collect_set, concat, size, coalesce, col, lpad, struct, count as sql_count, lit, min as sql_min, max as sql_max, collect_list, udf, when, desc, asc, to_date, create_map, sum as sql_sum
from pyspark.sql.types import StringType, ArrayType, MapType, StructType, StructField, IntegerType
from pyspark.sql.functions import array, regexp_extract
from itertools import chain
from churn.datapreparation.general.data_loader import get_unlabeled_car, get_port_requests_table, get_numclients_under_analysis
from churn.utils.constants import PORT_TABLE_NAME
from churn.utils.udf_manager import Funct_to_UDF
from pyspark.sql.functions import substring, datediff, row_number
from pykhaos.utils.date_functions import move_date_n_days, move_date_n_cycles
from pykhaos.utils.hdfs_functions import check_hdfs_exists
from pykhaos.modeling.model_performance import get_lift

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Functions

## Start spark context

In [4]:
from churn.utils.general_functions import init_spark
spark = init_spark("navcomp")
sc = spark.sparkContext

Ended spark session: 21.436152935 secs | default parallelism=4


# Tasa de evaporacion trigger nav comp

In [ ]:
tr_date_ = "20190925"
filter_ = 'comps'
save_ = True
verbose = True


from churn.analysis.triggers.navcomp.navcomp_utils import get_labeled_set_msisdn
from churn.analysis.triggers.base_utils.base_utils import get_active_filter, get_disconnection_process_filter, get_churn_call_filter

tr_set = get_labeled_set_msisdn(spark, tr_date_, sources='all', save_ = save_, verbose = verbose) 

# Modeling filters

tr_active_filter = get_active_filter(spark, tr_date_, 90)

tr_disconnection_filter = get_disconnection_process_filter(spark, tr_date_, 90)

tr_churn_call_filter = get_churn_call_filter(spark, tr_date_, 90, 'msisdn')

tr_set = tr_set \
    .join(tr_active_filter, ['msisdn'], 'inner') \
    .join(tr_disconnection_filter, ['nif_cliente'], 'inner') \
    .join(tr_churn_call_filter, ['msisdn'], 'inner')

tr_set = filter_population(spark, tr_set, filter_)

tr_set.columns


In [ ]:
[col_ for col_ in tr_set.columns if "port" in col_]

In [ ]:
df_test = tr_set

In [ ]:
competitors = ["PEPEPHONE", "ORANGE", "JAZZTEL", "MOVISTAR", "MASMOVIL", "YOIGO", "VODAFONE", "LOWI", "O2", "unknown"]

def get_most_consulted_operator(consulted_list, days_since_1st_navigation_list):
    if not consulted_list: return "None"
    combined_list = [(a,b,c) for a,b,c in zip(consulted_list, days_since_1st_navigation_list, competitors)]
    # sort by consulted list and then by days since 1st navigation
    sorted_list = sorted(combined_list, key = lambda x : (x[0], x[1]), reverse=True)
    return sorted_list[0][2] if sorted_list else "None"


get_most_consulted_operator_udf = udf(lambda x,y: get_most_consulted_operator(x,y), StringType())


df_test = (df_test
             .withColumn('consulted_list', array(*[col_ + "_sum_count" for col_ in competitors]))
             .withColumn('days_since_1st_navigation_list', array(*[col_ + "_max_days_since_navigation" for col_ in competitors]))
             .withColumn('most_consulted_operator', get_most_consulted_operator_udf(col('consulted_list'), col("days_since_1st_navigation_list"))))

#df_test.select(*([col_ + "_sum_count" for col_ in competitors] + [col_ + "_max_days_since_navigation" for col_ in competitors] +["most_consulted_operator"])).show()

In [ ]:
from churn_nrt.src.data.sopos_dxs import MobPort

df_target = MobPort(spark).get_module(tr_date_)


tr_set_target = tr_set.join(df_target, on=["msisdn"], how="left")
tr_set_churn = tr_set_target.where(col("label_mob")==1)

#tr_set_churn.select('portout_date_mob', "max_days_since_navigation_comps").show()

In [ ]:
#tr_set_churn = tr_set_churn.cache()
tr_set_churn.count()

In [ ]:
# days_until_churn: days since closing_day to churn_event (first churn event)
from pyspark.sql.functions import from_unixtime,unix_timestamp
# days_to_churn = fecha_de_sol_porta – fecha_de_la_última_navegación

# days_until_churn: days since closing_day to churn_event (first churn event)
tr_set_churn = (tr_set_churn.withColumn("days_until_sopo", when(col("portout_date_mob").isNotNull(), 
                                                                 datediff(col("portout_date_mob"),
                                                                          from_unixtime(unix_timestamp(lit(tr_date_), "yyyyMMdd")))).otherwise(-1)))

tr_set_churn = tr_set_churn.withColumn("days_until_churn", col("days_until_sopo") + col("min_days_since_navigation_comps"))



In [ ]:
col_ = "days_until_churn"
n = 20

bins, counts = tr_set_churn.select(col_).rdd.flatMap(lambda x: x).histogram(n)

plt.figure()
plt.hist(bins[:-1], bins=bins, weights=counts)
plt.xlabel("** {} **".format(col_))

total = tr_set_churn.count()

import numpy as np
counts_cum = np.cumsum(counts)
counts_cum = [100.0 * cc/total for cc in counts_cum]

plt.figure()
plt.hist(bins[:-1], bins=bins, weights=counts_cum)
plt.grid(True)
plt.yticks(list(range(10,110,10))) 
plt.xlabel("cumsum {}".format(col_))
plt.ylabel("% churners [total churners = {}]".format(total))

pd.DataFrame({"bins" : bins[1:], "counts" : counts_cum})

In [ ]:
base_df.unpersist()

from churn.analysis.triggers.base_utils.base_utils import get_customer_base
date_ = "20190930"
add_columns_customer = ["birth_date", "fecha_naci", 'CLASE_CLI_COD_CLASE_CLIENTE', 'X_CLIENTE_PRUEBA', "TIPO_DOCUMENTO"]
add_columns=["TARIFF", 'COD_ESTADO_GENERAL', "srv_basic"]
closing_day = date_
date_ = date_
base_df = (get_customer_base(spark, date_, add_columns=add_columns, 
                             add_columns_customer=add_columns_customer,
                            active_filter=False).filter(col('rgu')=='mobile'))#.select(*(['msisdn', 'nif_cliente', 'num_cliente'] + 
                                                                                                                                           #add_columns + add_columns_customer)))

print(base_df.columns)

base_df = (base_df.withColumn("birth_date", when( (col("birth_date").isNull() | (col("birth_date") == 1753)), col("fecha_naci")).otherwise(col("birth_date")).cast(IntegerType()))
                  .withColumn("age", when(col("birth_date") != 1753, lit(int(closing_day[:4])) - col("birth_date")).otherwise(-1))
          ).drop("birth_date")

base_df = base_df.cache()
base_df.columns

In [ ]:
base_df.groupby("srv_basic").agg(*[sql_count("*").alias("count")]).show()


In [ ]:
base_df.groupby("CLASE_CLI_COD_CLASE_CLIENTE").agg(*[sql_count("*").alias("count")]).show()


In [ ]:
base_df.select("X_CLIENTE_PRUEBA").dtypes

In [ ]:
base_df.groupby("X_CLIENTE_PRUEBA").agg(*[sql_count("*").alias("count")]).show()


In [ ]:
base_df.groupby("COD_ESTADO_GENERAL").agg(*[sql_count("*").alias("count")]).show()


In [ ]:
base_df.groupby("TARIFF").agg(*[sql_count("*").alias("count")]).show(100)


In [ ]:
base_df.select("NIF_CLIENTE").distinct().count()

In [ ]:
base_df.select("NUM_CLIENTE").distinct().count()


In [ ]:
base_df.select("msisdn").distinct().count()

In [ ]:
base_df.count()

In [ ]:
spark.read.load("/user/csanc109/data/mobile_base_20190930_parquet").count()

In [ ]:
base_df_insights.coalesce(1).write.mode('overwrite').save("/user/csanc109/data/mobile_base_20190930_parquet")

In [ ]:
base_df_insights = base_df.select("msisdn", "num_cliente", "nif_cliente", "srv_basic", "rgu", 'CLASE_CLI_COD_CLASE_CLIENTE', 'COD_ESTADO_GENERAL', "TARIFF")
base_df_insights.coalesce(1).write.mode('overwrite').format('csv').option('sep', '|').option('header', 'true').save("/user/csanc109/data/mobile_base_20190930")


In [ ]:
base_df  = base_df.withColumn("age_disc",  when(col("age").isNull(), "other")
                                         .when(col("age")<20, "<20")
                                         .when( (col("age")>=20) & (col("age")<25) , "[20-25)")
                                         .when( (col("age")>=25) & (col("age")<30) , "[25-30)")
                                         .when( (col("age")>=30) & (col("age")<35) , "[30-35)")
                                         .when( (col("age")>=35) & (col("age")<40) , "[35-40)")
                                         .when( (col("age")>=40) & (col("age")<45) , "[40-45)")
                                         .when( (col("age")>=45) & (col("age")<50) , "[45-50)")
                                         .when( (col("age")>=50) & (col("age")<55) , "[50-55)")
                                         .when( (col("age")>=55) & (col("age")<60) , "[55-60)")
                                         .when( (col("age")>=60) & (col("age")<65) , "[60-65)")
                                         .when( col("age")>=65, ">=65")
                                         .otherwise("other"))

base_df.groupby("age_disc").agg(sql_count("*").alias("count")).show()


In [ ]:
df_tgs.count()

In [ ]:
closing_day = "20191031"
df_tgs = spark.read.load("/data/udf/vf_es/churn/extra_feats_mod/tgs/year={}/month={}/day={}".format(int(closing_day[:4]), int(closing_day[4:6]), int(closing_day[6:])))
df_tgs.select('tgs_days_since_f_inicio_bi',
 'tgs_days_since_f_inicio_bi_exp',
 'tgs_days_until_f_fin_bi',
 'tgs_days_until_f_fin_bi_exp',
 'tgs_days_until_fecha_fin_dto').show()

In [ ]:
df_tgs.columns

In [ ]:


df_tgs = df_tgs.withColumn("blindaje_categ", when(  col("tgs_days_until_f_fin_bi")>60, "blindado")
                                      .when( (col("tgs_days_until_f_fin_bi")>0) & (col("tgs_days_until_f_fin_bi")<=60), "blindado proximo")
                                      .when( ((col("tgs_blindaje_bi_expirado")==1) & (col("tgs_days_since_f_inicio_bi_exp")>60)), "desblindado")
                                      .when( ((col("tgs_blindaje_bi_expirado")==1) & (col("tgs_days_since_f_inicio_bi_exp")>=0) & (col("tgs_days_since_f_inicio_bi_exp")<=60)), "desblindado reciente")
                    )

df_tgs.groupby("blindaje_categ").agg(sql_count("*").alias("count")).show()


In [ ]:

from churn.analysis.triggers.navcomp.navcomp_utils import get_latest_scores

df_scores = get_latest_scores(closing_day)

mean_score = df_scores.select(sql_avg('scoring').alias('mean_score')).rdd.first()['mean_score']


mean_score

# TESTING NEW ATTRIBUTES

In [ ]:
tr_date_ = "20191030"
filter_ = 'comps'
save_ = True
verbose = True


from churn.analysis.triggers.navcomp.navcomp_utils import get_labeled_set_msisdn
from churn.analysis.triggers.base_utils.base_utils import get_active_filter, get_disconnection_process_filter, get_churn_call_filter

tr_set = get_labeled_set_msisdn(spark, tr_date_, sources='all', save_ = save_, verbose = verbose) 
tr_set.columns

In [ ]:

def _get_customer_master_feats(customer_tr_df):

    class UDFclass:

        @staticmethod
        def compute_rgus_attrib(current_list, rgu_name):
            import collections
            rgus_dict = collections.Counter(current_list) if current_list is not None else {}
            if rgu_name not in rgus_dict:
                return 0
            else:
                return rgus_dict[rgu_name]

    compute_rgus_attrib_udf = udf(lambda x, y: UDFclass.compute_rgus_attrib(x, y), IntegerType())

    customer_tr_df = customer_tr_df.withColumn("nb_rgus_mobile", compute_rgus_attrib_udf("rgus_list", lit("mobile")))
    customer_tr_df = customer_tr_df.withColumn("nb_rgus_tv", compute_rgus_attrib_udf("rgus_list", lit("tv")))
    customer_tr_df = customer_tr_df.withColumn("nb_rgus_fixed", compute_rgus_attrib_udf("rgus_list", lit("fixed")))
    customer_tr_df = customer_tr_df.withColumn("nb_rgus_fbb", compute_rgus_attrib_udf("rgus_list", lit("fbb")))
    customer_tr_df = customer_tr_df.withColumn("nb_rgus_bam", compute_rgus_attrib_udf("rgus_list", lit("bam")))

    return customer_tr_df


In [ ]:
date_ = "20191022"

from churn.analysis.triggers.navcomp.metadata import get_metadata

from churn.analysis.triggers.base_utils.base_utils import get_customer_base
add_columns_customer = ["birth_date", "fecha_naci", 'CLASE_CLI_COD_CLASE_CLIENTE', 'X_CLIENTE_PRUEBA', "TIPO_DOCUMENTO"]
#add_columns = ["TARIFF", 'COD_ESTADO_GENERAL', "srv_basic"]

base_df = get_customer_base(spark, date_, add_columns_customer=add_columns_customer).filter(col('rgu')=='mobile').select('msisdn', 'nif_cliente', 'num_cliente', 'birth_date', 'fecha_naci')
base_df = base_df.drop_duplicates(['msisdn', 'nif_cliente', 'num_cliente'])
base_df = (base_df.withColumn("birth_date", when((col("birth_date").isNull() | (col("birth_date") == 1753)), col("fecha_naci")).otherwise(col("birth_date")).cast(IntegerType()))
                  .withColumn("age", when(col("birth_date") != 1753, lit(int(date_[:4])) - col("birth_date")).otherwise(-1)))
base_df = base_df.withColumn("age_disc", when(col("age").isNull(), "other").when(col("age") < 20, "<20")
                                        .when((col("age") >= 20) & (col("age") < 25), "[20-25)")
                                        .when((col("age") >= 25) & (col("age") < 30), "[25-30)")
                                        .when((col("age") >= 30) & (col("age") < 35), "[30-35)")
                                        .when((col("age") >= 35) & (col("age") < 40), "[35-40)")
                                        .when((col("age") >= 40) & (col("age") < 45), "[40-45)")
                                        .when((col("age") >= 45) & (col("age") < 50), "[45-50)")
                                        .when((col("age") >= 50) & (col("age") < 55), "[50-55)")
                                        .when((col("age") >= 55) & (col("age") < 60), "[55-60)")
                                        .when((col("age") >= 60) & (col("age") < 65), "[60-65)")
                                        .when(col("age") >= 65, ">=65").otherwise("other")).drop("birth_date")
# base_df.columns ['msisdn', 'nif_cliente', 'num_cliente', 'fecha_naci', 'age', 'age_disc']

from churn.analysis.triggers.orders.customer_master import get_customer_master

customer_metadata = get_metadata(spark, sources=['customer'])

customer_feats = customer_metadata.select('feature').rdd.map(lambda x: x['feature']).collect() + ['nif_cliente']



customer_tr_df = (get_customer_master(spark, date_, unlabeled=True)
                        .filter((col('segment_nif') != 'Pure_prepaid') & (col("segment_nif").isNotNull())).drop_duplicates(["nif_cliente"])
                )

customer_tr_df = (_get_customer_master_feats(customer_tr_df).select('nif_cliente', 
 'segment_nif',
 'nb_rgus',
 'rgus_list',
 'tgs_days_until_fecha_fin_dto',
 'nb_rgus_mobile',
 'nb_rgus_tv',
 'nb_rgus_fixed',
 'nb_rgus_fbb',
 'nb_rgus_bam'))

customer_map_tmp = customer_metadata.select("feature", "imp_value", "type").rdd.map(lambda x: (x["feature"], x["imp_value"], x["type"])).collect()

customer_map = dict([(x[0], str(x[1])) if x[2] == "categorical" else (x[0], float(x[1])) for x in customer_map_tmp])

base_df = base_df.join(customer_tr_df, ['nif_cliente'], 'inner').na.fill(customer_map)

base_df = base_df.select(customer_feats).na.fill(customer_map)



In [ ]:
base_df.count()

In [ ]:
spark.read.load("/data/udf/vf_es/churn/triggers/navcomp_msisdn_data/year=2019/month=11/day=7").columns

In [ ]:
df_mod_navcomp = spark.read.load("/data/attributes/vf_es/model_outputs/model_scores/model_name=triggers_navcomp/year=2019")
df_mod_navcomp.select("year", "month", "day", "predict_closing_date").distinct().show()

In [ ]:
customer_tr_df.select("msisdn", 'nb_rgus', 'rgus_list', 'tgs_days_until_fecha_fin_dto', 'nb_rgus_mobile', 'nb_rgus_tv', 'nb_rgus_fixed', 'nb_rgus_fbb', 'nb_rgus_bam')

In [ ]:
tt_date = "20190601"
from pykhaos.utils.date_functions import move_date_n_days
end_port = move_date_n_days(tt_date, 15)
from churn.analysis.triggers.base_utils.base_utils import get_mobile_portout_requests
target = get_mobile_portout_requests(spark, tt_date, end_port).withColumnRenamed('label_mob', 'label').select('msisdn', 'label', "portout_date_mob")
target.columns

# VANISHING RATE DEL TOP-k

In [ ]:
spark.read.load("/data/udf/vf_es/churn/triggers/nav_comp_tests/year=2019/month=11/day=7").columns

In [ ]:
[col_ for col_ in spark.read.load("/data/udf/vf_es/churn/triggers/navcomp_msisdn_data/year=2019/month=10/day=24").columns if "age" in col_]

In [ ]:
navcomp_date = "20190921"
target_date = navcomp_date

tt_date = -1 # "20190601"

navcomp_tr_df = spark.read.load("/data/udf/vf_es/churn/triggers/nav_comp_tests/year={}/month={}/day={}".format(int(navcomp_date[:4]), int(navcomp_date[4:6]), int(navcomp_date[6:]))).drop("label")

# Labeling

from churn.analysis.triggers.base_utils.base_utils import get_mobile_portout_requests

from pykhaos.utils.date_functions import move_date_n_days

end_port = move_date_n_days(target_date, 15)

target = get_mobile_portout_requests(spark, navcomp_date, end_port).withColumnRenamed('label_mob', 'label').select('msisdn', 'label', "portout_date_mob")

# Modeling filters

tt_active_filter = get_active_filter(spark, navcomp_date, 90)

tt_disconnection_filter = get_disconnection_process_filter(spark, navcomp_date, 90)

tt_churn_call_filter = get_churn_call_filter(spark, navcomp_date, 90, 'msisdn')


In [ ]:
tt_set.count()

In [ ]:
from churn.analysis.triggers.navcomp.navcomp_model import filter_population

tt_set = navcomp_tr_df \
    .join(tt_active_filter, ['msisdn'], 'inner') \
    .join(tt_disconnection_filter, ['nif_cliente'], 'inner') \
    .join(tt_churn_call_filter, ['msisdn'], 'inner')

tt_set = filter_population(spark, tt_set, filter_)

tr_set_target = tt_set.join(target, ['msisdn'], 'left').na.fill({'label': 0.0})

print "[Info navcomp_model_production] After all the filters - Sie of the test set: " + str(tt_set.count()) + " - Number of distinct MSISDNs in the test set: " + str(tt_set.select('msisdn').distinct().count())

tr_set_churn = tr_set_target.where(col("label")==1)

from pyspark.sql.functions import from_unixtime,unix_timestamp


In [ ]:
from churn.analysis.triggers.base_utils.base_utils import get_mobile_portout_requests


df_predictions = spark.read.load("/data/udf/vf_es/churn/triggers/nav_comp_tests_nav_comp_tests_alllabels/year=2019/month=9/day=21/")

target_date = "20190921"
navcomp_date = target_date

from pykhaos.utils.date_functions import move_date_n_days

end_port = move_date_n_days(target_date, 15)

target = get_mobile_portout_requests(spark, target_date, end_port).withColumnRenamed('label_mob', 'label').select('msisdn', 'label', "portout_date_mob")

tr_set_target = df_predictions.drop("label").join(target, ['msisdn'], 'left').na.fill({'label': 0.0})


In [ ]:
import math
num_nile = int(math.floor(float(tr_set_churn.count()) / 100.0))

lift = get_lift(tr_set_churn, 'scoring', 'label', num_nile, 1.0)



In [ ]:
[col_ for col_ in tr_set_target.columns if "scor" in col_]

In [ ]:
import pandas as pd
from pyspark.sql.functions import from_unixtime,unix_timestamp

col_ = "days_until_churn"
n = 20

myschema = tr_set_target.schema

total = tr_set_target.count()
print("total", total)

for top_ in [2000,3000,5000, 10000, 20000]:
    
    print("************** Analyzing top_{}".format(top_))
    
    tr_set_target = tr_set_target.sort(desc("model_score"))
    
    df_top = spark.createDataFrame(tr_set_target.head(top_), schema=myschema)
    
    tr_set_churn = df_top.where(col("label")==1)
    num_churners = tr_set_churn.count()
    
    tr_set_churn = (tr_set_churn.withColumn("days_until_sopo", when(col("portout_date_mob").isNotNull(), 
                                                                 datediff(col("portout_date_mob"), from_unixtime(unix_timestamp(lit(navcomp_date), "yyyyMMdd")))).otherwise(-1)))

    tr_set_churn = tr_set_churn.withColumn("days_until_churn", col("days_until_sopo") + col("min_days_since_navigation_comps"))

    print("1) num scores nulos", tr_set_churn.where(col("model_score").isNull()).count())

    tr_set_churn = tr_set_churn.where(col("model_score").isNotNull())
                                                
    tr_set_churn = tr_set_churn.cache()

    print("2) churn_rate top{} = {}".format(top_, 100.0*num_churners/top_))
    
    
    bins, counts = tr_set_churn.select(col_).rdd.flatMap(lambda x: x).histogram(n)

    plt.figure()
    plt.hist(bins[:-1], bins=bins, weights=counts)
    plt.xlabel("** {} ** (top = {}) (num_churners={}) (churn_rate={})".format(col_, top_, num_churners, 100.0*num_churners/top_))

    import numpy as np
    counts_cum = np.cumsum(counts)
    counts_cum = [100.0 * cc/num_churners for cc in counts_cum]
    
    plt.figure()
    plt.hist(bins[:-1], bins=bins, weights=counts_cum)
    plt.grid(True)
    plt.yticks(list(range(10,110,10))) 
    plt.xlabel("cumsum {} (top = {}) (num_churners={}) (churn_rate={})".format(col_, top_, num_churners, 100.0*num_churners/top_))

    print(top_, pd.DataFrame({"bins" : bins[1:], "counts" : counts_cum}))
    

In [ ]:
from churn.datapreparation.general.tgs_data_loader import get_tgs
from churn_nrt.src.utils.date_functions import get_previous_cycle

closing_day_tgs = get_previous_cycle(closing_day)
print("Getting tgs for date {}".format(closing_day_tgs))

df_tgs = get_tgs(spark, closing_day_tgs, closing_day_tgs[:6], impute_nulls=False).select("nif_cliente", 'tgs_days_until_fecha_fin_dto', 'tgs_days_until_f_fin_bi', 'msisdn')
#df_tgs.columns -- ['nif_cliente', 'tgs_days_until_fecha_fin_dto', 'tgs_days_until_f_fin_bi']


df_tgs = df_tgs.withColumn("blindaje_disc", when(col("tgs_days_until_f_fin_bi").isNull(), "none")
                             .when(col("tgs_days_until_f_fin_bi") > 60, "hard")
                             .when((col("tgs_days_until_f_fin_bi") >   0) & (col("tgs_days_until_f_fin_bi") <= 60), "soft")
                             .when((col("tgs_days_until_f_fin_bi") <=  0) & (col("tgs_days_until_f_fin_bi") >= -60), "soft-nobounded")
                             .otherwise("none"))

#df_tgs.groupby('tgs_days_until_f_fin_bi', "blindaje_disc").agg(sql_count("*").alias("count")).show()

In [ ]:
closing_day = "20191016"
date_ = closing_day
#from churn.analysis.triggers.orders.customer_master import get_tgs
from churn.datapreparation.general.tgs_data_loader import get_tgs
from churn_nrt.src.utils.date_functions import get_previous_cycle

closing_day_tgs = get_previous_cycle(closing_day)
print("Getting tgs for date {}".format(closing_day_tgs))

df_tgs = get_tgs(spark, closing_day_tgs, closing_day_tgs[:6], impute_nulls=False).select("nif_cliente", 'tgs_days_until_fecha_fin_dto', 'tgs_days_until_f_fin_bi', 'msisdn')
#df_tgs.columns -- ['nif_cliente', 'tgs_days_until_fecha_fin_dto', 'tgs_days_until_f_fin_bi']


df_tgs = df_tgs.withColumn("blindaje_disc", when(col("tgs_days_until_f_fin_bi").isNull(), "none")
                             .when(col("tgs_days_until_f_fin_bi") > 60, "hard")
                             .when((col("tgs_days_until_f_fin_bi") >   0) & (col("tgs_days_until_f_fin_bi") <= 60), "soft")
                             .when((col("tgs_days_until_f_fin_bi") <=  0) & (col("tgs_days_until_f_fin_bi") >= -60), "soft-nobounded")
                             .otherwise("none"))

# Desblindado [fcarren: Cliente desblindado donde ese desblindaje se produjo hace más de 2 meses, o que nunca tuvo blindaje]
# Desblindado reciente [fcarren: Cliente desblindado, pero su desblindaje se ha producido en los recientes 1 o 2 meses]
# Blindado próximo [fcarren: Cliente blindado, pero su le quedan sólo  1 o 2 meses para que se desblinde]
# Blindado [fcarren: Cliente blindado, con blindaje activo de más de 2 meses]



In [ ]:
df_tgs.groupby("blindaje_disc", "tgs_days_until_f_fin_bi").agg(sql_count("*").alias("count")).sort(asc("tgs_days_until_f_fin_bi")).show(300)

In [ ]:
df_tgs_raw.groupby("blindaje_disc").agg(sql_count("*").alias("count")).show()

In [ ]:
from churn.analysis.triggers.navcomp.navcomp_utils import get_navcomp_car_msisdn

df_navcomp = get_navcomp_car_msisdn(spark, date_, sources=["navcomp","customer","ccc","spinners","scores"], save_ = False, verbose = False)
nb_rgus_col = [col_ for col_ in df_navcomp.columns if col_.startswith("nb_rgus_")]

filter_ = ['comps']

from churn.analysis.triggers.base_utils.base_utils import get_active_filter, get_disconnection_process_filter, get_churn_call_filter


tt_active_filter = get_active_filter(spark, date_, 90)

tt_disconnection_filter = get_disconnection_process_filter(spark, date_, 90)

tt_churn_call_filter = get_churn_call_filter(spark, date_, 90, 'msisdn')

from churn.analysis.triggers.navcomp.navcomp_model import filter_population

tt_set = df_navcomp \
    .join(tt_active_filter, ['msisdn'], 'inner') \
    .join(tt_disconnection_filter, ['nif_cliente'], 'inner') \
    .join(tt_churn_call_filter, ['msisdn'], 'inner')




In [ ]:
#from churn.analysis.triggers.navcomp.navcomp_utils import get_volumen_attributes
#df_get_volumen = get_volumen_attributes(spark, date_, nDays=-14)#.select(volumen_feats)


In [ ]:
filter_ = 'comps'

tt_set = filter_population(spark, tt_set, filter_)

#tr_set_target = tt_set.join(target, ['msisdn'], 'left').na.fill({'label': 0.0})

In [ ]:
from churn.analysis.triggers.base_utils.base_utils import get_mobile_portout_requests

from pykhaos.utils.date_functions import move_date_n_days
end_port = move_date_n_days(date_, 15)
target = get_mobile_portout_requests(spark, date_, end_port).withColumnRenamed('label_mob', 'label').select('msisdn', 'label').withColumnRenamed("label", "label_1_15")

from pykhaos.utils.date_functions import move_date_n_days
end_port_2 = move_date_n_days(end_port, 15)
target_2 = get_mobile_portout_requests(spark, end_port, end_port_2).withColumnRenamed('label_mob', 'label').select('msisdn', 'label').withColumnRenamed("label", "label_16_30")


from pykhaos.utils.date_functions import move_date_n_days
end_port_3 = move_date_n_days(date_, 30)
target_3 = get_mobile_portout_requests(spark, date_, end_port_3).withColumnRenamed('label_mob', 'label').select('msisdn', 'label').withColumnRenamed("label", "label_1_30")


In [ ]:
date_,end_port,end_port_2, end_port_3

In [ ]:
from churn.analysis.triggers.orders.customer_master import get_customer_master
customer_tr_df = (get_customer_master(spark, date_, unlabeled=True).filter((col('segment_nif') != 'Pure_prepaid') & (col("segment_nif").isNotNull())).drop_duplicates(["nif_cliente"]))


In [ ]:
from churn.analysis.triggers.navcomp.navcomp_utils import get_latest_scores
df_scores = get_latest_scores(spark, date_)

In [ ]:
df_navcomp_select = tt_set.select("most_consulted_operator", "JAZZTEL_max_count", "VODAFONE_sum_count", "PEPEPHONE_min_days_since_navigation", "VODAFONE_min_days_since_navigation", 
                                      "norm_min_days_since_navigation_comps", "MOVISTAR_distinct_days_with_navigation", "unknown_distinct_days_with_navigation", "MOVISTAR_sum_count", 
                                      "norm_max_days_since_navigation_comps","msisdn", "age", "age_disc")

In [ ]:
#df_navcomp_select = df_navcomp_select.cache()
df_analysis.count()

In [ ]:
df_analysis = df_tgs.select(*['nif_cliente', 'tgs_days_until_fecha_fin_dto', 'tgs_days_until_f_fin_bi', 'blindaje_disc', 'msisdn']).join(df_navcomp_select, on=["msisdn"], how="inner")
df_analysis = df_analysis.join(df_scores, on=["msisdn"], how="left")
df_analysis = df_analysis.join(customer_tr_df.select("nif_cliente", "segment_nif"), ['nif_cliente'], 'inner')

df_analysis = df_analysis.join(target, ['msisdn'], 'left').na.fill({'label_1_15': 0.0})
df_analysis = df_analysis.join(target_2, ['msisdn'], 'left').na.fill({'label_16_30': 0.0})
df_analysis = df_analysis.join(target_3, ['msisdn'], 'left').na.fill({'label_1_30': 0.0})
#df_analysis.count()

In [ ]:
df_analysis.groupby("blindaje_disc").agg(sql_count("*").alias("count")).show()

In [ ]:
df_analysis = df_analysis.withColumn("age_disc_simple", when(col("age").isNull(), "other")
                             .when((col("age") >   0) & (col("age") < 20), "<20")
                             .when((col("age") >= 20) & (col("age") < 35), "[20-35)")
                             .when((col("age") >= 35) & (col("age") < 55), "[35-55)")
                             .when((col("age") >= 55) & (col("age") < 70), "[55-70)")
                             .when(col("age") >= 70, ">=70").otherwise("other"))

operadores = ['PEPEPHONE', 'JAZZTEL', 'MOVISTAR', 'MASMOVIL', 'YOIGO', 'VODAFONE', 'LOWI', 'O2', 'ORANGE']


for op in operadores:
    df_analysis = df_analysis.withColumn("flag_{}".format(op), when(col("most_consulted_operator")==op, 1).otherwise(0))

    
df_analysis = df_analysis.withColumn("most_consulted_operator_simple", when(col("most_consulted_operator").isin(["MOVISTAR", "VODAFONE", "YOIGO"]), col("most_consulted_operator")).otherwise("OTHERS"))


In [ ]:
from pyspark.sql.functions import avg as sql_avg
myschema = df_analysis.schema
df_analysis = df_analysis.sort(desc("scoring"))
df_top20k = spark.createDataFrame(df_analysis.head(20000), schema=myschema)



In [ ]:
df_top20k.groupby("segment_nif").agg(sql_count("*").alias("count")).show()

In [ ]:
df_top20k.groupby(['blindaje_disc']).agg(*([sql_count("*").alias("count"), sql_sum("label_1_15").alias("num_churners_1_15"),                  
                                                                                                             sql_avg("scoring").alias("avg_scoring")])).show(100,truncate=False)

In [ ]:
df_top20k.groupby("blindaje_disc").agg(sql_count("*").alias("count")).show()

In [ ]:
df_top20k.groupby(['age_disc', 'blindaje_disc', "most_consulted_operator", "segment_nif"]).agg(*([sql_count("*").alias("count"), sql_sum("label_1_15").alias("num_churners_1_15"),                  
                                                                                                             sql_avg("scoring").alias("avg_scoring")])).show(2500,truncate=False) 
                                                                                                            

In [ ]:

df_agg_top20k = df_top20k.groupby(['blindaje_disc', "most_consulted_operator", "segment_nif"]).agg(*([sql_count("*").alias("count"), 
                                                                                                             sql_sum("label_1_15").alias("num_churners_1_15"), 
                                                                                                             sql_sum("label_16_30").alias("num_churners_16_30"), 
                                                                                                             sql_sum("label_1_30").alias("num_churners_1_30"), 
                                                                                                             sql_avg("scoring").alias("avg_scoring")] + 
                                                                                                            [sql_sum("flag_{}".format(op)).alias("num_op_{}".format(op)) for op in operadores]))
df_agg_top20k.sort(desc("count")).show(5000, truncate=False)

In [ ]:
df_top20k.groupby(['age_disc_simple', "blindaje_disc"]).agg(*[sql_max("JAZZTEL_max_count").alias("MAX_JAZZTEL_max_count"),
                                                              sql_min("JAZZTEL_max_count").alias("MIN_JAZZTEL_max_count"), 
                                                              sql_avg("JAZZTEL_max_count").alias("AVG_JAZZTEL_max_count"), 
                                                              
                                                              sql_sum("VODAFONE_sum_count").alias("SUM_VODAFONE_sum_count"),
                                                              sql_avg("VODAFONE_sum_count").alias("AVG_VODAFONE_sum_count"),
                                                              sql_min("VODAFONE_sum_count").alias("MIN_VODAFONE_sum_count"),
                                                              sql_max("VODAFONE_sum_count").alias("MAX_VODAFONE_sum_count"),
                                                              
                                                              sql_min("PEPEPHONE_min_days_since_navigation").alias("MIN_PEPEPHONE_min_days_since_navigation"),
                                                              sql_max("PEPEPHONE_min_days_since_navigation").alias("MAX_PEPEPHONE_min_days_since_navigation"),
                                                              sql_avg("PEPEPHONE_min_days_since_navigation").alias("AVG_PEPEPHONE_min_days_since_navigation"),

                                                              sql_min("VODAFONE_min_days_since_navigation").alias("MIN_VODAFONE_min_days_since_navigation"),
                                                              sql_max("VODAFONE_min_days_since_navigation").alias("MAX_VODAFONE_min_days_since_navigation"),
                                                              sql_avg("VODAFONE_min_days_since_navigation").alias("AVG_VODAFONE_min_days_since_navigation"),

                                                              sql_min("norm_min_days_since_navigation_comps").alias("MIN_norm_min_days_since_navigation_comps"),
                                                              sql_max("norm_min_days_since_navigation_comps").alias("MAX_norm_min_days_since_navigation_comps"),
                                                              sql_count("norm_min_days_since_navigation_comps").alias("COUNT_norm_min_days_since_navigation_comps"),

                                                              sql_avg("MOVISTAR_distinct_days_with_navigation").alias("AVG_MOVISTAR_distinct_days_with_navigation"),
                                                              sql_min("MOVISTAR_distinct_days_with_navigation").alias("MIN_MOVISTAR_distinct_days_with_navigation"),
                                                              sql_max("MOVISTAR_distinct_days_with_navigation").alias("MAX_MOVISTAR_distinct_days_with_navigation"),

                                                              sql_avg("unknown_distinct_days_with_navigation").alias("AVG_unknown_distinct_days_with_navigation"),
                                                              sql_min("unknown_distinct_days_with_navigation").alias("MIN_unknown_distinct_days_with_navigation"),
                                                              sql_max("unknown_distinct_days_with_navigation").alias("MAX_unknown_distinct_days_with_navigation"),
                                                              
                                                              sql_avg("MOVISTAR_sum_count").alias("AVG_MOVISTAR_sum_count"),
                                                              sql_min("MOVISTAR_sum_count").alias("MIN_MOVISTAR_sum_count"),
                                                              sql_max("MOVISTAR_sum_count").alias("MAX_MOVISTAR_sum_count"),

                                                              sql_max("norm_max_days_since_navigation_comps").alias("MAX_norm_max_days_since_navigation_comps"),
                                                              sql_min("norm_max_days_since_navigation_comps").alias("MIN_norm_max_days_since_navigation_comps"),
                                                              sql_avg("norm_max_days_since_navigation_comps").alias("AVG_norm_max_days_since_navigation_comps"),

                                                              sql_count("*").alias("count"), 
                                                                                                                          
                                                             ]).show()

In [ ]:
df_agg_2 = df_top20k.groupby(['blindaje_disc', "most_consulted_operator"]).agg(*[sql_count("*").alias("count"), sql_avg("scoring").alias("avg_scoring")])

In [ ]:
df_agg_2.sort(desc("count")).show(50, truncate=False)

In [ ]:
df_agg_3 = df_analysis.groupby(["most_consulted_operator", 'age_disc_simple']).agg(*[sql_count("*").alias("count"), sql_avg("scoring").alias("avg_scoring")])

In [ ]:
df_agg_3.sort(desc("count")).show(150, truncate=False)

In [ ]:
from churn_nrt.src.projects_utils.analysis.exploratory_analysis import attributes_exploration

df_predictions = spark.read.load("/data/udf/vf_es/churn/triggers/nav_comp_tests_nav_comp_tests_alllabels/year=2019/month=9/day=21/")

target_date = "20190921"

dfSalidaGaus = attributes_exploration(spark, df_predictions, ['age', 'latest_score'] + nb_rgus_col, percentile_inc=5)

dfSalidaGaus.show()

In [ ]:
from churn_nrt.src.projects_utils.analysis.exploratory_analysis import attributes_exploration

df_predictions_2 = spark.read.load("/data/udf/vf_es/churn/triggers/nav_comp_tests_all_labels/year=2019/month=8/day=15/time=1574273309")

dfSalidaGaus_2 = attributes_exploration(spark, df_predictions_2, ['age', 'latest_score'] + nb_rgus_col, percentile_inc=5)

dfSalidaGaus_2.show()




In [ ]:
df_model_scores = spark.read.load("/user/csanc109/data/mobile_base_20190930_parquet")
df_model_scores.columns

In [ ]:
df_model_scores.columns

In [ ]:

df_model_scores = spark.read.load("/user/csanc109/data/mobile_base_20190930_parquet")
df_model_scores = df_model_scores.withColumn("scoring", lit(0.0))
df_model_scores = df_model_scores.withColumnRenamed("nif_cliente", "nif")
df_model_scores = df_model_scores.withColumnRenamed("num_cliente", "client_id")


from pyspark.sql.functions import length, concat_ws, regexp_replace, split
from churn.datapreparation.general.model_outputs_manager import ensure_types_model_scores_columns

EXTRA_INFO_COLS = ['srv_basic',
 'rgu',
 'CLASE_CLI_COD_CLASE_CLIENTE',
 'COD_ESTADO_GENERAL',
 'TARIFF']

MODEL_OUTPUTS_NULL_TAG = ""

executed_at = dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S ")

return_feed_execution = "20190930"

day_partition = int(return_feed_execution[6:])
month_partition = int(return_feed_execution[4:6])
year_partition = int(return_feed_execution[:4])
training_closing_date = "20190930"

print("Going to insert with partition {} {} {} ".format(year_partition, month_partition, day_partition))

'''
MODEL PARAMETERS
'''

import pandas as pd
df_pandas = pd.DataFrame({
    "model_name": ["mobile_base"],
    "executed_at": [executed_at],
    "model_level": ["msisdn"],
    "training_closing_date": [training_closing_date if training_closing_date else closing_day],
    "target": [""],
    "model_path": [""],
    "metrics_path": [""],
    "metrics_train": [""],
    "metrics_test": [""],
    "varimp": [""],
    "algorithm": [""],
    "author_login": ["csanc109"],
    "extra_info": [""],
    "scores_extra_info_headers": [";".join(EXTRA_INFO_COLS)],
    "year":  [year_partition],
    "month": [month_partition],
    "day":   [day_partition],
    "time": [int(executed_at.split(" ")[1].replace(":", ""))]
})

df_parameters = spark.createDataFrame(df_pandas).withColumn("day", col("day").cast("integer"))\
                                                .withColumn("month", col("month").cast("integer"))\
                                                .withColumn("year", col("year").cast("integer"))\
                                                .withColumn("time", col("time").cast("integer"))

'''
MODEL SCORES
'''

# set to null the columns that go to extra_info field and have no value
for col_ in EXTRA_INFO_COLS:
    df_model_scores = df_model_scores.withColumn(col_,
                                                 when(coalesce(length(col(col_)), lit(0)) == 0, MODEL_OUTPUTS_NULL_TAG).otherwise(
                                                     col(col_)))

df_model_scores = (
        df_model_scores.withColumn("extra_info", concat_ws(";", *[col(col_name) for col_name in EXTRA_INFO_COLS]))
        .withColumn("prediction", lit("0"))
        .drop(*EXTRA_INFO_COLS)
        .withColumnRenamed("comb_score", "scoring")
        .withColumn("model_name", lit("mobile_base"))
        .withColumn("executed_at", lit(executed_at))
        .withColumn("model_executed_at", lit(executed_at))
        .withColumn("year", lit(year_partition).cast("integer"))
        .withColumn("month",lit(month_partition).cast("integer"))
        .withColumn("day", lit(day_partition).cast("integer"))
        .withColumn("time", regexp_replace(split(col("executed_at"), " ")[1], ":", "").cast("integer"))
        .withColumn("predict_closing_date", lit("20190930"))
        .withColumn("model_output", lit(""))
    )

df_model_scores = ensure_types_model_scores_columns(df_model_scores)



In [ ]:
# save_path = "/user/csanc109/data/mobile_base/model_scores"

# (df_model_scores.coalesce(1)
#   .write
#   .partitionBy('model_name', 'year', 'month', 'day')
#   .mode("append")
#   .format("parquet")
#   .save(save_path))

In [ ]:
day_path1 = "/data/udf/vf_es/churn/triggers/nav_comp_tests_all_labels/year=2019/month=10/day=21/time=1574782575"
day_path2 = "/data/udf/vf_es/churn/triggers/nav_comp_tests_all_labels/year=2019/month=10/day=22/time=1574791124"
day_path3 = "/data/udf/vf_es/churn/triggers/nav_comp_tests_all_labels/year=2019/month=10/day=23/time=1574863576"
day_path4 = "/data/udf/vf_es/churn/triggers/nav_comp_tests_all_labels/year=2019/month=10/day=24/time=1574791742"
day_path5 = "/data/udf/vf_es/churn/triggers/nav_comp_tests_all_labels/year=2019/month=10/day=25/time=1574886360"
day_path6 = "/data/udf/vf_es/churn/triggers/nav_comp_tests_all_labels/year=2019/month=10/day=26/time=1574887287"
day_path7 = "/data/udf/vf_es/churn/triggers/nav_comp_tests_all_labels/year=2019/month=10/day=27/time=1574888086"

df_labels = spark.read.load(day_path2)
#label_kind = "mobile"
label_kind = "mobile+fix" 
closing_day = "20191022"


print("NUM LABELS ORIG", df_labels.select(sql_sum('label').alias('num_churners')).rdd.first()['num_churners'])

if label_kind in  ["mobile+fix"]:
    df_lab = get_label(spark, closing_day, kind=label_kind, churn_window=15).select("msisdn", "label")
    df_labels = df_labels.drop("label")
    df_labels = df_labels.join(df_lab, on=["msisdn"], how="left").na.fill({'label': 0.0})
elif label_kind != "mobile":
    print("Unknown kind {}. Program will exit here!".format(kind))
    import sys
    sys.exit()

from pyspark.sql.functions import avg as sql_avg
myschema = df_labels.schema
df_labels = df_labels.sort(desc("model_score"))
df_top_9286 = spark.createDataFrame(df_labels.head(9286), schema=myschema)

tt_churn_ref = df_top_9286.select(sql_avg('label').alias('churn_ref')).rdd.first()['churn_ref']
tt_churn_ref * 100

In [ ]:
df_labels_A = spark.read.load(day_path5)
df_labels_B = spark.read.load(day_path7)
df_labels_join = df_labels_A.select("msisdn").join(df_labels_B.select("msisdn", "label", "model_score"), ['msisdn'], 'right').where(df_labels_A['msisdn'].isNull())
df_labels_join = df_labels_join.cache()
df_labels_join.count()

In [ ]:
tt_churn_ref = df_labels_join.select(sql_avg('label').alias('churn_ref')).rdd.first()['churn_ref']
tt_churn_ref

In [ ]:
myschema3 = df_labels_join.schema
from churn_nrt.src.projects_utils.models.modeler import get_cumulative_lift_fix_step
cum_churn_rate = get_cumulative_lift_fix_step(spark, df_labels_join, ord_col =  'model_score', label_col = 'label', step_=500)
#cum_churn_rate.show(150, truncate=False)

# Referencia de Insights

In [ ]:
from churn_nrt.src.utils.date_functions import move_date_n_days
from pyspark.sql.functions import avg as sql_avg


msisdn_df = spark.read.table("raw_es.campaign_msisdncontacthist")
closing_day = "20191023"
label_kind = "mobile+fix"


insights_df = msisdn_df.filter((col('year')==int(closing_day[:4])) & (col('month') == int(closing_day[4:6])) & (col('day') == int(closing_day[6:])) & (col("CampaignCode").contains("COMP_WEB")) & (col("Creatividad").startswith("LLAM_"))).select("msisdn").distinct()
print("clients", insights_df.count())
#start_port = move_date_n_days(closing_day, n=1)
end_port = move_date_n_days(closing_day, n=15)

from churn.analysis.triggers.base_utils.base_utils import get_mobile_portout_requests, get_customer_base
#insights_df.join(get_mobile_portout_requests(spark, start_port, end_port).select("msisdn", "label_mob").withColumnRenamed("label_mob", "label"), ['msisdn'], 'left').na.fill({'label': 0.0}).select(sql_avg('label')).show()
insights_df.join(get_label(spark, closing_day, kind=label_kind, churn_window=15).select("msisdn", "label"), ['msisdn'], 'left').na.fill({'label': 0.0}).select(sql_avg('label')).show()


In [ ]:
msisdn_df = spark.read.table("raw_es.campaign_msisdncontacthist")
msisdn_df = msisdn_df.filter((col("CampaignCode").contains("COMP_WEB")) & (col("Creatividad").startswith("LLAM_"))).drop_duplicates(["msisdn"])
msisdn_df = msisdn_df.where(col("year")==2019).where(col("month")>=10)

msisdn_df.groupby("year", "month", "day").agg(sql_count("*").alias("count")).sort(desc("year"), desc("month"), desc("day")).show()

In [ ]:

def get_label(spark, closing_day, kind="mobile", churn_window=15):
    from churn_nrt.src.utils.date_functions import move_date_n_days
    #start_port = move_date_n_days(closing_day, n=1)
    end_port = move_date_n_days(closing_day, n=churn_window)

    if kind=="mobile":

        from churn.analysis.triggers.base_utils.base_utils import get_mobile_portout_requests, get_customer_base
        df_target = get_mobile_portout_requests(spark, closing_day, end_port).select("msisdn", "label_mob").withColumnRenamed("label_mob", "label")
        return df_target
    
    elif kind=="mobile+fix":
        
        print("Computing target with mobile and fix sopo")
        

        from churn_nrt.src.data.sopos_dxs import FixPort
        from churn_nrt.src.data.customer_base import CustomerBase

        # Getting portout requests for fix and mobile services, and disconnections of fbb services
        print("******* Asking for FixPort...")
        df_sopo_fix = FixPort(spark).get_module(closing_day, save=False, churn_window=churn_window)

        # The base of active services on closing_day
        from churn.analysis.triggers.base_utils.base_utils import get_mobile_portout_requests, get_customer_base
        base_df = get_customer_base(spark, closing_day).filter(col('rgu') == 'mobile').select('msisdn', "nif_cliente")
        
        base_df = base_df\
                 .join(get_mobile_portout_requests(spark, closing_day, end_port).select('msisdn', 'label_mob'), ['msisdn'], 'left').na.fill(0.0)

        df_sopos = (base_df.join(df_sopo_fix, ['msisdn'], "left").na.fill({'label_srv': 0.0}))

        # 1 if any of the services of this nif is 1
        window_nc = Window.partitionBy("nif_cliente")

        df_sopos = df_sopos.withColumn('tmp', when((col('label_srv') == 1.0) | (col('label_mob') == 1.0), 1.0).otherwise(0.0))
        df_sopos = df_sopos.withColumn('label', sql_max('tmp').over(window_nc)).drop("tmp").na.fill({'label': 0.0})     
        
        return df_sopos
    
    else:
        print("Unknown kind {}. Program will exit here!".format(kind))
        return None
                
        
#get_label(spark, closing_day="20191024", kind="mobile")

In [ ]:
df_labels = spark.read.load(day_path2)
#label_kind = "mobile"
label_kind = "mobile" 
closing_day = "20191022"

print("NUM LABELS ORIG", df_labels.select(sql_sum('label').alias('num_churners')).rdd.first()['num_churners'])

if label_kind in  ["mobile+fix", "mobile"]:
    df_lab = get_label(spark, closing_day, kind=label_kind, churn_window=15).select("msisdn", "label")
    df_labels = df_labels.drop("label")
    df_labels = df_labels.join(df_lab, on=["msisdn"], how="left").na.fill({'label': 0.0})
elif label_kind != "mobile":
    print("Unknown kind {}. Program will exit here!".format(kind))
    import sys
    sys.exit()

#from pyspark.sql.functions import avg as sql_avg
#myschema = df_labels.schema
#df_labels = df_labels.sort(desc("model_score"))
#df_top_9286 = spark.createDataFrame(df_labels.head(9286), schema=myschema)

num_churners = df_labels.select(sql_sum('label').alias('num_churners')).rdd.first()['num_churners']
num_churners

In [ ]:
from churn_nrt.src.data.navcomp_data import NavCompData


df_navcomp_data = NavCompData(spark).get_module("20191016")

In [ ]:
spark.read.load("/data/udf/vf_es/churn_nrt/customer_base/year=2019/month=10/day=16").columns

In [ ]:
starting_date = "20191001"
process_date = "20191016"
level= "msisdn"
orig_path = True

from pyspark.sql.functions import (col,
                                    when,
                                    lit,
                                    lower,
                                    array,
                                    concat,
                                    translate,
                                    count,
                                    sum as sql_sum,
                                    max as sql_max,
                                    min as sql_min,
                                    avg as sql_avg,
                                    lpad,
                                    greatest,
                                    least,
                                    isnull,
                                    isnan,
                                    struct,
                                    substring,
                                    size,
                                    length,
                                    udf,
                                    year,
                                    month,
                                    dayofmonth,
                                    unix_timestamp,
                                    date_format,
                                    from_unixtime,
                                    datediff,
                                    to_date,
                                    desc,
                                    asc,
                                    countDistinct,
                                    row_number,
                                    regexp_replace,
                                    upper,
                                    trim)



from functools import reduce
import itertools
from pykhaos.utils.date_functions import get_diff_days

if("nif" in level.lower()):
    level = "nif_cliente"

print("[Info get_navcomp_attributes] starting_date: " + starting_date + " - ending_date: " + process_date + " - level: " + level)

window_length = get_diff_days(starting_date, process_date)

print("[Info get_navcomp_attributes] Starting the computation of the feats")

competitors = ["PEPEPHONE", "ORANGE", "JAZZTEL", "MOVISTAR", "MASMOVIL", "YOIGO", "VODAFONE", "LOWI", "O2", "unknown"]

count_feats = [p[0] + "_" + p[1] for p in list(itertools.product(competitors, ["sum_count", "max_count"]))]

days_feats = [p[0] + "_" + p[1] for p in list(itertools.product(competitors, ["max_days_since_navigation", "min_days_since_navigation", "distinct_days_with_navigation"]))]

null_imp_dict = dict([(x, 0) for x in count_feats] + [(x, 0) for x in days_feats if(("distinct_days" in x) | ("max_days" in x))] + [(x, 10000) for x in days_feats if("min_days" in x)])

from churn.analysis.triggers.base_utils.base_utils import get_customer_base

basedf = get_customer_base(spark, process_date).select('msisdn', 'nif_cliente')
#print("BASEDF {}".format(basedf.count()))

path_to_read = "/data/attributes/vf_es/return_feed/data_navigation/" if orig_path==True else "/data/udf/vf_es/netscout/dailyMSISDNApplicationName/"
apps_names = [u'WEB_O2_HTTPS',
             u'WEB_MOVISTAR_HTTPS',
             u'WEB_VODAFONE_HTTPS',
             u'WEB_YOIGO_HTTP',
             u'WEB_LOWI_HTTPS',
             u'WEB_ORANGE_HTTP',
             u'WEB_JAZZTEL_HTTP',
             u'WEB_MASMOVIL_HTTP',
             u'WEB_VODAFONE_HTTP',
             u'WEB_MOVISTAR_HTTP',
             u'WEB_ORANGE_HTTPS',
             u'WEB_PEPEPHONE_HTTPS',
             u'WEB_PEPEPHONE_HTTP',
             u'WEB_JAZZTEL_HTTPS',
             u'WEB_LOWI_HTTP',
             u'WEB_MASMOVIL_HTTPS',
             u'WEB_YOIGO_HTTPS',
             u'WEB_O2_HTTP']

print("Reading from {}".format(path_to_read))

repart_navigdf = (spark.read.parquet(path_to_read))

if not orig_path:
    print("Adding event_date")
    repart_navigdf = (repart_navigdf.where((col("SUM_userplane_upload_bytes_count") + col("SUM_userplane_download_bytes_count")) > 524288)
                                    .where(col("application_name").isin(apps_names))
                                    .withColumn("event_date",  concat(col('year'), lit("-"), lpad(col('month'), 2, '0'), lit("-"), lpad(col('day'), 2, '0'))))

repart_navigdf = (repart_navigdf.withColumn("formatted_event_date", from_unixtime(unix_timestamp(col("event_date"), "yyyy-MM-dd")))
        .filter((col("formatted_event_date") >= from_unixtime(unix_timestamp(lit(starting_date), "yyyyMMdd"))) & (col("formatted_event_date") <= from_unixtime(unix_timestamp(lit(process_date), "yyyyMMdd"))))
        .withColumn("days_since_navigation", datediff(from_unixtime(unix_timestamp(lit(process_date), "yyyyMMdd")), col("formatted_event_date")).cast("double"))
        .withColumn("msisdn", col("subscriber_msisdn").substr(3, 9))
        .select("msisdn", "application_name", "count", "formatted_event_date", "days_since_navigation")
  )

print("hola1", repart_navigdf.count())

repart_navigdf = (repart_navigdf
        .withColumn("competitor", lit("unknown"))
        .withColumn("competitor", when(col("application_name").contains("PEPEPHONE"), "PEPEPHONE").otherwise(col("competitor")))
        .withColumn("competitor", when(col("application_name").contains("ORANGE"), "ORANGE").otherwise(col("competitor")))
        .withColumn("competitor", when(col("application_name").contains("JAZZTEL"), "JAZZTEL").otherwise(col("competitor")))
        .withColumn("competitor", when(col("application_name").contains("MOVISTAR"), "MOVISTAR").otherwise(col("competitor")))
        .withColumn("competitor", when(col("application_name").contains("MASMOVIL"), "MASMOVIL").otherwise(col("competitor")))
        .withColumn("competitor", when(col("application_name").contains("YOIGO"), "YOIGO").otherwise(col("competitor")))
        .withColumn("competitor", when(col("application_name").contains("VODAFONE"), "VODAFONE").otherwise(col("competitor")))
        .withColumn("competitor", when(col("application_name").contains("LOWI"), "LOWI").otherwise(col("competitor")))
        .withColumn("competitor", when(col("application_name").contains("O2"), "O2").otherwise(col("competitor")))
        .join(basedf, ['msisdn'], 'inner')
        .repartition(400)
)

print("hola2", repart_navigdf.count())


def get_most_consulted_operator(consulted_list, days_since_1st_navigation_list):
    if not consulted_list: return "None"
    combined_list = [(a, b, c) for a, b, c in zip(consulted_list, days_since_1st_navigation_list, competitors)]
    # sort by consulted list and then by days since 1st navigation
    sorted_list = sorted(combined_list, key=lambda x: (x[0], x[1]), reverse=True)
    return sorted_list[0][2] if sorted_list else "None"

get_most_consulted_operator_udf = udf(lambda x, y: get_most_consulted_operator(x, y), StringType())

#print("[Info get_navcomp_attributes] After joining the base - Volume of repart_navigdf: " + str(repart_navigdf.count()) + " - Distinct msisdn: " + str(repart_navigdf.select('msisdn').distinct().count()) + " - Distinct nif: " + str(repart_navigdf.select('nif_cliente').distinct().count()))

repart_navigdf = (
    repart_navigdf
        .groupBy(level)
        .pivot("competitor", competitors)
        .agg(sql_sum("count").alias("sum_count"),
             sql_max("count").alias("max_count"),
             sql_max("days_since_navigation").alias("max_days_since_navigation"),
             sql_min("days_since_navigation").alias("min_days_since_navigation"),
             countDistinct("formatted_event_date").alias("distinct_days_with_navigation"))
        .na
        .fill(null_imp_dict)
        .withColumn("sum_count_vdf", reduce(lambda a, b: a + b, [col(x) for x in count_feats if(("sum_count" in x) & ("VODAFONE" in x))]))
        .withColumn("sum_count_comps", reduce(lambda a, b: a + b, [col(x) for x in count_feats if (("sum_count" in x) & ("VODAFONE" not in x))]))
        .withColumn("num_distinct_comps", reduce(lambda a, b: a + b, [when(col(x) > 0, 1.0).otherwise(0.0) for x in count_feats if(("sum_count" in x) & ("VODAFONE" not in x))]))
        .withColumn("max_count_comps", greatest(*[x for x in count_feats if (("max_count" in x) & ("VODAFONE" not in x))]))
        .withColumn("sum_distinct_days_with_navigation_vdf", reduce(lambda a, b: a + b, [col(x) for x in days_feats if (("distinct_days_with_navigation" in x) & ("VODAFONE" in x))]))
        .withColumn("norm_sum_distinct_days_with_navigation_vdf", col("sum_distinct_days_with_navigation_vdf").cast('double')/lit(window_length).cast('double'))
        .withColumn("sum_distinct_days_with_navigation_comps", reduce(lambda a, b: a + b, [col(x) for x in days_feats if (("distinct_days_with_navigation" in x) & ("VODAFONE" not in x))]))
        .withColumn("norm_sum_distinct_days_with_navigation_comps", col("sum_distinct_days_with_navigation_comps").cast('double') / lit(window_length).cast('double'))
        .withColumn("min_days_since_navigation_comps", least(*[x for x in days_feats if (("min_days_since_navigation" in x) & ("VODAFONE" not in x))]))
        .withColumn("norm_min_days_since_navigation_comps", col("min_days_since_navigation_comps").cast('double') / lit(window_length).cast('double'))
        .withColumn("max_days_since_navigation_comps", greatest(*[x for x in days_feats if (("max_days_since_navigation" in x) & ("VODAFONE" not in x))]))
        .withColumn("norm_max_days_since_navigation_comps", col("max_days_since_navigation_comps").cast('double') / lit(window_length).cast('double'))
        .withColumn('consulted_list', array(*[col_ + "_sum_count" for col_ in competitors]))
        .withColumn('days_since_1st_navigation_list',  array(*[col_ + "_max_days_since_navigation" for col_ in competitors]))
        .withColumn('most_consulted_operator', get_most_consulted_operator_udf(col('consulted_list'), col("days_since_1st_navigation_list")))
        .drop('consulted_list', 'days_since_1st_navigation_list'))

#print("[Info get_navcomp_attributes] After pivoting - Volume of repart_navigdf: " + str(repart_navigdf.count()) + " - Distinct " + level + ": " + str(repart_navigdf.select(level).distinct().count()))

print("[Infoget_navcomp_attributes] Competitors web extraction completed")


In [ ]:
repart_navigdf.count()

In [ ]:
target_2.columns

In [ ]:

closing_day = "20190714"
from churn_nrt.src.data.customer_base import CustomerBase
base_df = CustomerBase(spark).get_module(closing_day)
base_df.columns


In [ ]:
from churn_nrt.src.data.customers_data import Customer
customer_df = Customer(spark).get_module(closing_day, save=False)
customer_df.columns

In [ ]:
spark.read.load("/data/udf/vf_es/churn_nrt/ccc/year=2019/month=7/day=14").columns

In [ ]:
closing_day = "20190714"
from churn_nrt.src.data.ccc import CCC
tr_ccc_all = CCC(spark, level="nif").get_module(closing_day)

In [ ]:
tr_ccc_all.columns

In [ ]:
import os
logging_file = os.path.join("/var/SP/data/bdpmdses/deliveries_churn/logging",
                                "churn_delivery_log_time_" + dt.datetime.now().strftime("%Y%m%d_%H%M%S") + ".log")
logging_file